In [1]:
import crocoddyl
import casadi as cs
import numpy as np
from models.cartpole import ActionModelCartpole

class ActionModelCasadi(crocoddyl.ActionModelAbstract):
    """
    A custom Crocoddyl action model that uses CasADi for automatic differentiation.
    """

    def __init__(self, state, nu, cost_weights, dt, m_cart, m_pole, l_pole, grav):
        """
        Initialize the CasADi action model.

        Args:
            state (crocoddyl.StateAbstract): The state representation.
            nu (int): The dimension of the control input.
            cost_weights (list): A list of weights for the cost function.
            dt (float): The time step for integration.
            m_cart (float): The mass of the cart.
            m_pole (float): The mass of the pole.
            l_pole (float): The length of the pole.
            grav (float): The gravitational acceleration.
        """
        crocoddyl.ActionModelAbstract.__init__(self, state, nu)

        # System parameters
        self.Δt = dt
        self.m_cart = m_cart
        self.m_pole = m_pole
        self.l_pole = l_pole
        self.grav = grav
        self.costWeights = cost_weights

        # Define CasADi symbolic variables
        x_cas = cs.SX.sym("x", self.state.nx)
        u_cas = cs.SX.sym("u", self.nu)

        y, θ, ẏ, θ̇ = x_cas[0], x_cas[1], x_cas[2], x_cas[3]
        f = u_cas[0]

        # Shortname for system parameters
        sin_θ, cos_θ = cs.sin(θ), cs.cos(θ)

        # Cartpole dynamics
        μ = self.m_cart + self.m_pole * sin_θ * sin_θ
        ÿ = f / μ + self.m_pole * self.grav * cos_θ * sin_θ / μ - self.m_pole * self.l_pole * (sin_θ * θ̇ ** 2 / μ)
        θ̈ = (f / self.l_pole) * cos_θ / μ + ((self.m_cart + self.m_pole) * self.grav / self.l_pole) * sin_θ / μ - self.m_pole * cos_θ * sin_θ * θ̇**2 / μ

        # Forward Euler integration
        ẏ_next = ẏ + self.Δt * ÿ
        θ̇_next = θ̇ + self.Δt * θ̈
        y_next = y + self.Δt * ẏ_next
        θ_next = θ + self.Δt * θ̇_next

        xnext = cs.vertcat(y_next, θ_next, ẏ_next, θ̇_next)

        # Cost function
        cost_r = self.costWeights * cs.vertcat(sin_θ, 1.0 - cos_θ, y, ẏ, θ̇, f)
        cost = 0.5 * cs.sumsqr(cost_r)

        # Create CasADi functions
        self.dynamics_func = cs.Function('dyn', [x_cas, u_cas], [xnext])
        self.cost_func = cs.Function('cost', [x_cas, u_cas], [cost])
        self.cost_r_func = cs.Function('cost_r', [x_cas, u_cas], [cost_r])

        # Derivatives
        self.Fx = cs.Function('Fx', [x_cas, u_cas], [cs.jacobian(xnext, x_cas)])
        self.Fu = cs.Function('Fu', [x_cas, u_cas], [cs.jacobian(xnext, u_cas)])
        self.Lx = cs.Function('Lx', [x_cas, u_cas], [cs.jacobian(cost, x_cas)])
        self.Lu = cs.Function('Lu', [x_cas, u_cas], [cs.jacobian(cost, u_cas)])
        self.Lxx = cs.Function('Lxx', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, x_cas), x_cas)])
        self.Luu = cs.Function('Luu', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, u_cas), u_cas)])
        self.Lxu = cs.Function('Lxu', [x_cas, u_cas], [cs.jacobian(cs.jacobian(cost, x_cas), u_cas)])


    def calc(self, data, x, u=None):
        """
        Computes the next state and the cost function.
        """
        if u is None:
            u = np.zeros(self.nu)

        data.xnext = np.asarray(self.dynamics_func(x, u)).flatten()
        data.cost = np.asarray(self.cost_func(x, u)).item()
        data.r = np.asarray(self.cost_r_func(x,u)).flatten()

        return data.xnext, data.cost

    def calcDiff(self, data, x, u=None):
        """
        Computes the derivatives of the dynamics and the cost function.
        """
        if u is None:
            u = np.zeros(self.nu)

        # Compute derivatives using CasADi functions
        data.Fx = np.asarray(self.Fx(x, u))
        data.Fu = np.asarray(self.Fu(x, u))
        data.Lx = np.asarray(self.Lx(x, u)).flatten()
        data.Lu = np.asarray(self.Lu(x, u)).flatten()
        data.Lxx = np.asarray(self.Lxx(x, u))
        data.Luu = np.asarray(self.Luu(x, u))
        data.Lxu = np.asarray(self.Lxu(x, u))


if __name__ == '__main__':
    # Define state and action model
    state = crocoddyl.StateVector(4)
    nu = 1
    
    # System parameters
    dt = 0.01
    m_cart = 1.0
    m_pole = 0.1
    l_pole = 0.5
    grav = 9.81
    
    # Cost weights
    cost_weights = np.array([1.0, 1.0, 0.1, 0.001, 0.001, 1.0])
    
    actuation = ActionModelCasadi(state, nu, cost_weights, dt, m_cart, m_pole, l_pole, grav)
    model = ActionModelCartpole()

    # Create data structures
    action_data = actuation.createData()
    model_data = model.createData()
    
    # Define an initial state and control
    x0 = np.array([2., 2, 1., 3.])
    u0 = np.array([0.])
    
    # Compute and display the dynamics and cost
    actuation.calc(action_data, x0, u0)
    model.calc(model_data, x0, u0)
    
    print("Next state:", action_data.xnext)
    print("Next state 2:", model_data.xnext)
    print("Cost:", action_data.cost)
    print("Cost 2:", model_data.cost)
    
    # Compute and display the derivatives
    actuation.calcDiff(action_data, x0, u0)
    model.calcDiff(model_data, x0, u0)
    print("Fx:\n", action_data.Fx)
    print("Fu:\n", action_data.Fu)
    print("Lx:", action_data.Lx)
    print("Lu:", action_data.Lu)
    print("=========")
    print("Fx:\n", model_data.Fx)
    print("Fu:\n", model_data.Fu)
    print("Lx:", model_data.Lx)
    print("Lu:", model_data.Lu)

Next state: [2.00992792 2.03184403 0.99279202 3.18440331]
Next state 2: [2.00992792 2.03184403 0.99279202 3.18440331]
Cost: 1.4361518365471424
Cost 2: 1.4361518365471424
Fx:
 [[ 1.00000000e+00 -4.69674775e-05  1.00000000e-02 -2.51956883e-05]
 [ 0.00000000e+00  9.99353695e-01  0.00000000e+00  1.00209702e-02]
 [ 0.00000000e+00 -4.69674775e-03  1.00000000e+00 -2.51956883e-03]
 [ 0.00000000e+00 -6.46305353e-02  0.00000000e+00  1.00209702e+00]]
Fu:
 [ 9.23632085e-05 -7.68733140e-05  9.23632085e-03 -7.68733140e-03]
Lx: [2.00000000e-02 9.09297427e-01 1.00000000e-06 3.00000000e-06]
Lu: [0.]
Fx:
 [[ 1.00000000e+00 -4.69674775e-05  1.00000000e-02 -2.51956883e-05]
 [ 0.00000000e+00  9.99353695e-01  0.00000000e+00  1.00209702e-02]
 [ 0.00000000e+00 -4.69674775e-03  1.00000000e+00 -2.51956883e-03]
 [ 0.00000000e+00 -6.46305353e-02  0.00000000e+00  1.00209702e+00]]
Fu:
 [ 9.23632085e-05 -7.68733140e-05  9.23632085e-03 -7.68733140e-03]
Lx: [2.00000000e-02 9.09297427e-01 1.00000000e-06 3.00000000e-06]

In [2]:
# ... (the previous __main__ block)

# Define the OCP
x0 = np.array([0.0, 0.0, 1.0, 0.5])
T = 100 # Number of knots

# Create a list of action models for the OCP
running_models = [model] * T
#terminal_model = ActionModelCasadi(state, nu, cost_weights * 10, dt, m_cart, m_pole, l_pole, grav) # Higher terminal cost
terminal_model = model

problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)

# Create a solver
solver = crocoddyl.SolverDDP(problem)
solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

# Solve the OCP
xs = [x0] * (T + 1)
us = [np.zeros(nu)] * T
solver
solver.solve(xs, us)


iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi


True

   0  2.406e+01  0.000e+00  3.335e+01  6.669e+01  1.000e-09  1.000e-09  1.0000  1.000e-02  0.000e+00  0.000e+00  3.335e+01 -2.406e+01  0.000e+00  0.000e+00
   1  2.404e+01  0.000e+00  2.662e-02  5.324e-02  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  2.662e-02  2.868e-02  0.000e+00  0.000e+00
   2  2.404e+01  0.000e+00  1.963e-04  3.926e-04  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  1.963e-04  2.112e-04  0.000e+00  0.000e+00
   3  2.404e+01  0.000e+00  1.145e-06  2.291e-06  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  1.145e-06  1.232e-06  0.000e+00  0.000e+00
   4  2.404e+01  0.000e+00  6.525e-09  1.305e-08  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  6.525e-09  7.017e-09  0.000e+00  0.000e+00
   5  2.404e+01  0.000e+00  3.710e-11  7.419e-11  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  3.710e-11  3.988e-11  0.000e+00  0.000e+00


In [3]:
# ... (the previous __main__ block)

# Define the OCP
x0 = np.array([0.0, 0.0, 1.0, 0.5])
T = 100 # Number of knots

# Create a list of action models for the OCP
running_models = [actuation] * T
#terminal_model = ActionModelCasadi(state, nu, cost_weights * 10, dt, m_cart, m_pole, l_pole, grav) # Higher terminal cost
terminal_model = actuation

problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)

# Create a solver
solver = crocoddyl.SolverDDP(problem)
solver.setCallbacks([crocoddyl.CallbackLogger(), crocoddyl.CallbackVerbose()])

# Solve the OCP
xs = [x0] * (T + 1)
us = [np.zeros(nu)] * T
solver
solver.solve(xs, us)


iter    cost       merit      stop      |grad|      preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  2.406e+01  0.000e+00  3.335e+01  6.669e+01  1.000e-09  1.000e-09  1.0000  1.000e-02  0.000e+00  0.000e+00  3.335e+01 -2.406e+01  0.000e+00  0.000e+00
   1  2.404e+01  0.000e+00  2.662e-02  5.324e-02  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  2.662e-02  2.868e-02  0.000e+00  0.000e+00
   2  2.404e+01  0.000e+00  1.963e-04  3.926e-04  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  1.963e-04  2.112e-04  0.000e+00  0.000e+00
   3  2.404e+01  0.000e+00  1.145e-06  2.291e-06  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  1.145e-06  1.232e-06  0.000e+00  0.000e+00
   4  2.404e+01  0.000e+00  6.525e-09  1.305e-08  1.000e-09  1.000e-09  1.0000  0.000e+00  0.000e+00  0.000e+00  6.525e-09  7.017e-09  0.000e+00  0.000e+00
   5  2.404e+01  0.000e+00  3.710e-11  7.419e-11  1.000e-09  1.000e

True

0.000e+00  0.000e+00  0.000e+00  3.710e-11  3.988e-11  0.000e+00  0.000e+00
